## Gradient-based Inverse Design Maximize Qfwd while Minimize Qback
---

### <u> Import modules <u/>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from wgangp_model import load_generator

2025-02-27 14:46:24.047837: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-27 14:46:24.055723: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-27 14:46:24.064361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-27 14:46:24.066915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 14:46:24.074043: I tensorflow/core/platform/cpu_feature_guar

## Set dynamic GPU memory growth

In [2]:
# Check if GPU available
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU is available and set to memory growth mode.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected by TensorFlow.")

GPU is available and set to memory growth mode.


I0000 00:00:1740663985.867105    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740663985.891134    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740663985.892601    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


### Define the own resblock class

keras requires custom classes to be defined for being able to reload

In [3]:
# decorator to register the custom resblock to allow serialziation and re-loading
@keras.utils.register_keras_serializable()  # for keras3
class ResBlock1D(keras.Model):
    def __init__(self, filters, kernel_size=3, convblock=False, **kwargs):
        super(ResBlock1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

        # setup all necessary layers
        self.conv1 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn1 = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()

        # self.relu = keras.layers.LeakyReLU()
        self.relu = keras.layers.LeakyReLU(negative_slope=0.01)

        self.convblock = convblock
        if self.convblock:
            self.conv_shortcut = keras.layers.Conv1D(filters, 1)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        # add shortcut. optionally pass it through a Conv
        if self.convblock:
            x_sc = self.conv_shortcut(input_tensor)
        else:
            x_sc = input_tensor
        x += x_sc
        return self.relu(x)

    def get_config(self):
        base_config = super().get_config()
        return {
            "convblock": self.convblock,
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            **base_config,
        }

## Reload the forward and wgangp model

In [4]:
forward_path = "models/resnet_Mie_predictor.keras"
wgangp_path = "models/wgangp_generator.h5"

forward_model = keras.models.load_model(forward_path)
generator = load_generator(wgangp_path)

I0000 00:00:1740663987.559543    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740663987.561686    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740663987.562934    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1740663987.660649    4441 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

## Define Target


In [5]:
wavelengths = np.linspace(400, 800, 64)
print(f"wavelengths:{wavelengths}")
target_lambda_index = np.argmin(np.abs(wavelengths - 705))

print(
    f"Target wavelength index {target_lambda_index} ({wavelengths[target_lambda_index]} nm)"
)

wavelengths:[400.         406.34920635 412.6984127  419.04761905 425.3968254
 431.74603175 438.0952381  444.44444444 450.79365079 457.14285714
 463.49206349 469.84126984 476.19047619 482.53968254 488.88888889
 495.23809524 501.58730159 507.93650794 514.28571429 520.63492063
 526.98412698 533.33333333 539.68253968 546.03174603 552.38095238
 558.73015873 565.07936508 571.42857143 577.77777778 584.12698413
 590.47619048 596.82539683 603.17460317 609.52380952 615.87301587
 622.22222222 628.57142857 634.92063492 641.26984127 647.61904762
 653.96825397 660.31746032 666.66666667 673.01587302 679.36507937
 685.71428571 692.06349206 698.41269841 704.76190476 711.11111111
 717.46031746 723.80952381 730.15873016 736.50793651 742.85714286
 749.20634921 755.55555556 761.9047619  768.25396825 774.6031746
 780.95238095 787.3015873  793.65079365 800.        ]
Target wavelength index 48 (704.7619047619048 nm)


## Maximize Qfwd Minimize Qback Fitness function
---
Add Series Weight

In [6]:
def objective_function_weight_series(
    z_batch, generator, forward_model, target_lambda_index, weight_Qback
):
    synthetic_geometries = generator(z_batch)
    synthetic_geometries_concat = keras.ops.concatenate(synthetic_geometries, axis=1)

    # Forward pass through the forward model
    predicted_batch = forward_model(synthetic_geometries_concat)

    # Split output from the forward model
    predicted_Qfwd_batch = predicted_batch[..., 0]  # Qfwd is the first channel
    predicted_Qback_batch = predicted_batch[..., 1]  # Qback is the second channel

    # Extract values at the target wavelength index
    fitness_fwd = -predicted_Qfwd_batch[:, target_lambda_index]  # Maximize Qfwd
    fitness_back = predicted_Qback_batch[:, target_lambda_index] * weight_Qback
    # Combine to get total loss
    total_loss_batch = fitness_fwd + fitness_back
    return total_loss_batch


In [7]:
def optimize_latent_vector_weight_series_parallel(
    z_batch,
    generator,
    forward_model,
    target_lambda_index,
    weight_Qback,
    initial_lr=0.01,
    iterations=250,
    learning_rates=None,
):

    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    loss_history = []

    for i in range(iterations):
        with tf.GradientTape() as tape:
            # Calculate the loss for the batch of latent vectors
            total_loss_batch = objective_function_weight_series(
                z_batch, generator, forward_model, target_lambda_index, weight_Qback
            )
        gradients_batch = tape.gradient(total_loss_batch, [z_batch])
        optimizer.apply_gradients(zip(gradients_batch, [z_batch]))

        mean_loss = tf.reduce_mean(total_loss_batch).numpy()
        loss_history.append(total_loss_batch.numpy())

        if i % 50 == 0:
            print(f"Iteration {i}, Mean Loss: {mean_loss:.6f}")

    final_loss = tf.reduce_mean(total_loss_batch).numpy()

    return z_batch, final_loss, loss_history

## Running Optimization Weight Series

In [8]:
# Initialize variables
batch_size = 500
latent_dim = 128

weight_series = [0, 0.5, 1, 5]

initial_learning_rate = 0.01
iterations = 250
results = []

In [9]:
# Loop through each weight in the weight series
for weight_Qback in weight_series:
    print(f"\nRunning optimization with lr= {initial_learning_rate}, weight {weight_Qback}")

    initial_z_batch = np.random.normal(size=(batch_size, latent_dim)) * 2
    z_batch_tf = tf.Variable(initial_z_batch, dtype=tf.float32)

    start_time = time.time()

    # Optimize the latent vectors for the current weight
    optimized_z_batch, final_loss, loss_history = (
        optimize_latent_vector_weight_series_parallel(
            z_batch_tf,
            generator,
            forward_model,
            target_lambda_index,
            weight_Qback,
            initial_lr=initial_learning_rate,
            iterations=iterations,
        )
    )

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Store the optimized latent vectors and final loss for this weight
    results.append(
        {
            "weight_Qback": weight_Qback,
            "learning_rate": initial_learning_rate,
            "optimized_z_batch": optimized_z_batch,
            "final_loss": final_loss,
            "loss_history": loss_history,
        }
    )

# Print summary of results
for result in results:
    print(
        f"\nWeight: {result['weight_Qback']}, Learning Rate: {result['learning_rate']}"
    )
    print(f"Final Loss: {result['final_loss']:.4f}")


Running optimization with lr= 0.01, weight 0


2025-02-27 14:47:37.953171: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1740664057.986849    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.011394    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.014828    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.015720    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.015903    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.016063    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.016225    4441 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1740664058.017056    4441 gpu_t

Iteration 0, Mean Loss: 0.375496
Iteration 50, Mean Loss: -0.327635
Iteration 100, Mean Loss: -0.376180
Iteration 150, Mean Loss: -0.389565
Iteration 200, Mean Loss: -0.400385

Running optimization with lr= 0.01, weight 0.5
Iteration 0, Mean Loss: 0.106357
Iteration 50, Mean Loss: -0.552752
Iteration 100, Mean Loss: -0.587534
Iteration 150, Mean Loss: -0.604939
Iteration 200, Mean Loss: -0.617200

Running optimization with lr= 0.01, weight 1
Iteration 0, Mean Loss: -0.200811
Iteration 50, Mean Loss: -0.870363
Iteration 100, Mean Loss: -0.917348
Iteration 150, Mean Loss: -0.937626
Iteration 200, Mean Loss: -0.951856

Running optimization with lr= 0.01, weight 5
Iteration 0, Mean Loss: -2.548327
Iteration 50, Mean Loss: -4.017037
Iteration 100, Mean Loss: -4.078430
Iteration 150, Mean Loss: -4.100632
Iteration 200, Mean Loss: -4.131439

Weight: 0, Learning Rate: 0.01
Final Loss: -0.4033

Weight: 0.5, Learning Rate: 0.01
Final Loss: -0.6222

Weight: 1, Learning Rate: 0.01
Final Loss: -0.9

### Save Results

In [11]:
with open("best_geometries/gradient_max_min_problem_lrfixed.pkl", "wb") as pickle_file:
    pickle.dump(results, pickle_file)